In [1]:
import os
import glob
import numpy as np 
import xarray as xr
import pandas as pd
import datetime
from datetime import date, timedelta
import dask

# Plotting utils 
import matplotlib
import matplotlib.pyplot as plt 
import cartopy
import cartopy.crs as ccrs


In [ ]:
# # Grabbed from Brian M. to use time midpoints, not end periods
# def cesm_correct_time(ds):
#     """Given a Dataset, check for time_bnds,
#        and use avg(time_bnds) to replace the time coordinate.
#        Purpose is to center the timestamp on the averaging inverval.   
#        NOTE: ds should have been loaded using `decode_times=False`
#     """
#     assert 'time_bnds' in ds
#     assert 'time' in ds
#     correct_time_values = ds['time_bnds'].mean(dim='nbnd')
#     # copy any metadata:
#     correct_time_values.attrs = ds['time'].attrs
#     ds = ds.assign_coords({"time": correct_time_values})
#     ds = xr.decode_cf(ds)  # decode to datetime objects
#     return ds

In [6]:
file_path = '/glade/campaign/cesm/development/cross-wg/S2S/land/rest/2021-06-01-00000/'
!ls /glade/campaign/cesm/development/cross-wg/S2S/land/rest/2021-06-01-00000

I2000Clm50BgcCrop.002runRealtime.cism.r.2021-06-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.clm2.h0.2020-09-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.clm2.h0.2021-01-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.clm2.h1.2021-05-29-00000.nc
I2000Clm50BgcCrop.002runRealtime.clm2.h1.2021-05-30-00000.nc
I2000Clm50BgcCrop.002runRealtime.clm2.r.2021-06-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.clm2.rh0.2021-06-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.clm2.rh1.2021-06-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.cpl.r.2021-06-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.mosart.h0.2021-05.nc
I2000Clm50BgcCrop.002runRealtime.mosart.h1.2021-05-29-00000.nc
I2000Clm50BgcCrop.002runRealtime.mosart.h1.2021-05-31-00000.nc
I2000Clm50BgcCrop.002runRealtime.mosart.r.2021-06-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.mosart.rh0.2021-06-01-00000.nc
I2000Clm50BgcCrop.002runRealtime.mosart.rh1.2021-06-01-00000.nc
rpointer.atm
rpointer.drv
rpointer.glc
rpointer.lnd
rpointer.rof


In [7]:
file1 = file_path+'I2000Clm50BgcCrop.002runRealtime.clm2.r.2021-06-01-00000.nc'
file2 = file_path+'I2000Clm50BgcCrop.002runRealtime.clm2.rh0.2021-06-01-00000.nc'
file3 = file_path+'I2000Clm50BgcCrop.002runRealtime.clm2.rh1.2021-06-01-00000.nc'

ds1 = xr.open_dataset(file1)
ds2 = xr.open_dataset(file2)
ds3 = xr.open_dataset(file3)


/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_nstep_rad_prev' has multiple fill values {-9999, -999999999} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_type' has multiple fill values {-9999, -999999999} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_step_sec' has multiple fill values {-9999, -999999999} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_start_ymd' has multiple fill values {-9999, -999999999}

## Test 1: increase initial soil moisture arbitrarily 


In [43]:
test_highSM_file = '/glade/derecho/scratch/mdfowler/temp/out_mult2.nc'
test_highDS = xr.open_dataset(test_highSM_file)

/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_nstep_rad_prev' has multiple fill values {-9999, -999999999} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_type' has multiple fill values {-9999, -999999999} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_step_sec' has multiple fill values {-9999, -999999999} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'timemgr_rst_start_ymd' has multiple fill values {-9999, -999999999}

In [35]:
ds_restart_lowSM = ds1.copy(deep=True)
ds_restart_lowSM['H2OSOI_LIQ'] = ds_restart_lowSM['H2OSOI_LIQ'] * 0.5

In [36]:
ds_restart_highSM = ds1.copy(deep=True)
ds_restart_highSM['H2OSOI_LIQ'] = ds_restart_highSM['H2OSOI_LIQ'] * 2

In [46]:
np.nanmin(test_highDS['H2OSOI_LIQ'] - ds_restart_highSM['H2OSOI_LIQ'])

0.0

## Investigating

In [13]:
ds1.H2OSOI_LIQ

<xarray.DataArray 'H2OSOI_LIQ' (column: 141928, levtot: 37)> Size: 42MB
[5251336 values with dtype=float64]
Dimensions without coordinates: column, levtot
Attributes:
    long_name:                 liquid water
    units:                     kg/m2
    interpinic_flag:           1
    interpinic_flag_meanings:  1=nearest neighbor, 2=copy directly, 3=skip, 4...
    varnames_on_old_files:     H2OSOI_LIQ
    switchdim_flag:            1
    switchdim_flag_values:     [0 1]
    switchdim_flag_is_0:       1st and 2nd dims are same as model representation
    switchdim_flag_is_1:       1st and 2nd dims are switched from model repre...

In [14]:
fsurdat = '/glade/campaign/cesm/cesmdata/inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr2000_c190214.nc'
finidat = '/glade/campaign/cesm/cesmdata/inputdata/lnd/clm2/initdata_map/clmi.I2000Clm50BgcCrop.2011-01-01.1.9x2.5_gx1v7_gl4_simyr2000_c190312.nc'
params =  '/glade/campaign/cesm/cesmdata/inputdata/lnd/clm2/paramdata/clm5_params.c171117.nc'

ds_fsurdat = xr.open_dataset(fsurdat)
ds_finidat = xr.open_dataset(finidat) 
ds_params  = xr.open_dataset(params) 


In [24]:
ds_fsurdat

<xarray.Dataset> Size: 493MB
Dimensions:                  (lsmlat: 192, lsmlon: 288, nlevsoi: 10,
                              natpft: 15, cft: 2, time: 12, lsmpft: 17,
                              numurbl: 3, numrad: 2, nlevurb: 5, nglcecp1: 11,
                              nglcec: 10)
Coordinates:
  * natpft                   (natpft) int32 60B 0 1 2 3 4 5 ... 9 10 11 12 13 14
  * cft                      (cft) int32 8B 15 16
  * time                     (time) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lsmlat, lsmlon, nlevsoi, lsmpft, numurbl,
                                numrad, nlevurb, nglcecp1, nglcec
Data variables: (12/81)
    mxsoil_color             int32 4B ...
    SOIL_COLOR               (lsmlat, lsmlon) int32 221kB ...
    PCT_SAND                 (nlevsoi, lsmlat, lsmlon) float64 4MB ...
    PCT_CLAY                 (nlevsoi, lsmlat, lsmlon) float64 4MB ...
    ORGANIC                  (nlevsoi, lsmlat, lsmlon) float64 4MB ...
    FMAX                     (lsmlat, lsmlon) float64 442kB 0.3902 ... 0.3658
    ...                       ...
    CONST_HARVEST_SH2        (lsmlat, lsmlon) float64 442kB ...
    CONST_HARVEST_SH3        (lsmlat, lsmlon) float64 442kB ...
    CONST_GRAZING            (lsmlat, lsmlon) float64 442kB ...
    CONST_FERTNITRO_CFT      (cft, lsmlat, lsmlon) float64 885kB ...
    UNREPRESENTED_PFT_LULCC  (natpft, lsmlat, lsmlon) float64 7MB ...
    UNREPRESENTED_CFT_LULCC  (cft, lsmlat, lsmlon) float64 885kB ...
Attributes: (12/50)
    Conventions:                          NCAR-CSM
    History_Log:                          created on: 02-14-19 10:18:19
    Logname:                              erik
    Host:                                 cheyenne3
    Source:                               Community Land Model: CLM5
    Version:                              release-clm5.0.18/gpfs/fs1/scratch/...
    ...                                   ...
    map_vic_file:                         map_0.9x1.25_GRDC_to_0.9x1.25_nomas...
    Soil_texture_raw_data_file_name:      mksrf_soitex.10level.c010119.nc
    Soil_color_raw_data_file_name:        mksrf_soilcolor_CMIP6_simyr2005.c17...
    Fmax_raw_data_file_name:              mksrf_fmax_3x3min_USGS_c120911.nc
    Organic_matter_raw_data_file_name:    mksrf_organic_10level_5x5min_ISRIC-...
    Vegetation_type_raw_data_filename:    mksrf_landuse_histclm50_LUH2_2000.c...

In [25]:
ds_fsurdat.FMAX

<xarray.DataArray 'FMAX' (lsmlat: 192, lsmlon: 288)> Size: 442kB
array([[0.390196, 0.390196, 0.390196, ..., 0.390196, 0.390196, 0.390196],
       [0.390196, 0.390196, 0.390196, ..., 0.390196, 0.390196, 0.390196],
       [0.390196, 0.390196, 0.390196, ..., 0.390196, 0.390196, 0.390196],
       ...,
       [0.365783, 0.365783, 0.365783, ..., 0.365783, 0.365783, 0.365783],
       [0.365783, 0.365783, 0.365783, ..., 0.365783, 0.365783, 0.365783],
       [0.365783, 0.365783, 0.365783, ..., 0.365783, 0.365783, 0.365783]])
Dimensions without coordinates: lsmlat, lsmlon
Attributes:
    long_name:  maximum fractional saturated area
    units:      unitless

In [33]:
np.nanmin(ds_finidat.H2OSOI_LIQ.values)


-4.0245584642661925e-16